<div style="text-align: center;"><br>
<img src="https://assets-global.website-files.com/62b9d45fb3f64842a96c9686/62d84db4aeb2f6552f3a2f78_Quantinuum%20Logo__horizontal%20blue.svg" width="200" height="200" /></div>

# Saving and Reloading `ResultHandle` to Monitor Job Status

This document shows two workflows to use reference data (`pytket.backends.resulthandle.ResultHandle`) for H-Series jobs across different sessions.

A H-Series job submission with pytket involves:
1. circuit synthesis.
2. compilation to the H-Series gate-set.
3. submission of the circuit to H-Series (the job).
4. retrieval of job results.

Completion of step 3. produces a [`pytket.backends.resulthandle.ResultHandle`](https://tket.quantinuum.com/api-docs/backends.html#pytket.backends.resulthandle.ResultHandle). This `ResultHandle` is required to complete step 4. Sometimes, a job may take hours or days to run, depending on demand for the device, operational hours (visible in the user portal), the fair-share queue and the customer's group- and user-priorities.

In this scenario, it is recommended to save the `ResultHandle` data to disk after step 3. is completed and reload from disk once step 4. is possible.

**Content**
* [Save `ResultHandle` to Disk](#Save-ResultHandle-to-Disk)
* [Reload `ResultHandle` from Disk](#Reload-ResultHandle-from-Disk)
* [Reload `ResultHandle` using Job ID](#reload-resulthandle-using-Job-ID)

## Save `ResultHandle` to Disk

The `ResultHandle` is created after a circuit is submitted with QuantinuumBackend to H-Series.

In [1]:
from pytket.extensions.quantinuum import QuantinuumBackend

backend = QuantinuumBackend(device_name="H1-1SC")
backend.login()

In [2]:
from pytket.circuit import Circuit
from numpy.random import random_sample

circ = Circuit(10)
for qubit in circ.qubits:
    angles = random_sample(2) * 0.1
    circ.PhasedX(*angles.tolist(), qubit)

measurement_circuit = circ.copy().measure_all()

In [3]:
result_handle = backend.process_circuit(
    measurement_circuit, n_shots=10, options={"tket-opt-level": 2}
)

In [4]:
file_io = open("result_handles.txt", "w")
file_io.write(str(result_handle))
file_io.close()

## Reload `ResultHandle` from Disk

The `ResultHandle` data is read from disk and reloaded into the `ResultHandle` object using the [`from_str`](https://tket.quantinuum.com/api-docs/backends.html#pytket.backends.resulthandle.ResultHandle.from_str) method.

In [5]:
text_stream = open("result_handles.txt", "r")
result_handle_text = text_stream.read()
text_stream.close()

In [6]:
from pytket.backends.resulthandle import ResultHandle

reloaded_result_handle = ResultHandle.from_str(result_handle_text)
print(reloaded_result_handle)

('93ab1856386641acaa0a55f3a70239c6', 'null', 10, '[["c", 0], ["c", 1], ["c", 2], ["c", 3], ["c", 4], ["c", 5], ["c", 6], ["c", 7], ["c", 8], ["c", 9]]')


This can now be used to retrieve job results from H-Series.

In [7]:
result = backend.get_result(reloaded_result_handle)

In [8]:
print(result.get_distribution())

{(0, 0, 0, 0, 0, 0, 0, 0, 0, 0): 1.0}


## Reload `ResultHandle` using Job ID

It is also possible to retrieve H-Series data using the Job ID alone. However, it is recommended to always save `ResultHandle` information to disk using the workflow above.

All Job IDs are recorded in the user portal. A single Job ID is the first argument of `ResultHandle`. The second and fourth argument are the string "null" and the third argument is an integer, which is set to the integer 0 in the code-cell below.

In [9]:
handle_from_id = ResultHandle("66f2da298ea84373b2b7968a7bb6d7b7", "null", 0, "null")

In [10]:
result_from_id = backend.get_result(handle_from_id)

QuantinuumAPIError: HTTP error attempting: job status.

Server Response: {'error': {'code': 21, 'text': 'Error: job does not exist'}}

In [ ]:
print(result_from_id.get_distribution())

{(): 1.0}


## Summary

This document shows two workflows to use reference data (`pytket.backends.resulthandle.ResultHandle`) for H-Series jobs across different sessions.
1. Save and reload from disk.
2. Reinstantiate using Job ID.

This workflow is very useful when jobs are submitted and results are recovered on different days. It is recommended to use method 1 over method 2.